In [2]:
%matplotlib
import numpy as np
from matplotlib import pyplot as plt
import signal_tools as st
from urQRd import urQRd
from scipy.linalg import norm

Using matplotlib backend: Qt5Agg


In [3]:
def params(s,n,Ps,Pn):
    '''
    s : nb dim peaks sig
    n : nb dim noise
    Ps : power sig
    Pn : power noise
    '''
    a = s/Ps
    b = n/Pn
    k = a-b
    l = s+n #-200
    return a,b,k,l

# Analytical solution

In [4]:
class ANALYT(object):
    '''
    Analytical solution
    '''
    def __init__(self):
        self.x = np.arange(l)
    @property
    def sig(self):
        return 1-((s*a+n*b+k*(l-self.x)) - np.sqrt((s*a+n*b+k*(l-self.x))**2 - 4*k*s*a*(l-self.x)))/(2*s*k)
    @property
    def noise(self):
        #return 1-b*self.sig/(a-self.sig*k)
        #return b*self.sig/(a-self.sig*k)
        return 1-((-n*b-s*a+k*(l-self.x)) + np.sqrt((-n*b-s*a+k*(l-self.x))**2 + 4*k*n*b*(l-self.x)))/(2*n*k)

# Numerical solution

In [5]:
class NUMER(object):
    '''
    Numerical solution
    '''
    def __init__(self):
        self.sig = []                   # signal
        self.noise = []                 # noise
        self.xis = self.xin = 0
        self.calc()
    def calc(self):
        for i in range(l):
            empty = (Ps*(1-self.xis)**2 + Pn*(1 - self.xin)**2)         # power not yet retrieved.
            self.xis += Ps*(1-self.xis)**2/empty/s                      # part of signal dimension retrieved
            self.xin += Pn*(1-self.xin)**2/empty/n                      # part of noise dimension retrieved
            self.sig.append(self.xis)
            self.noise.append(self.xin)
        self.sig = np.array(self.sig)
        self.noise = np.array(self.noise)

# Comparison analytical and numerical

# Make the signal

In [57]:
nbpeaks = 10
lenfid = 4000
noise = 15
ampl = 70
lampl = list(ampl*np.ones(nbpeaks))
sig = st.SIGNAL_NOISE( lenfid= lenfid, nbpeaks = nbpeaks, 
          amplitude = lampl,   # *np.random.randn(nbpeaks)
          shape='list', noise = noise)
nlevel = st.findnoiselevel_offset(sig.spec, nbseg=50)
print(nlevel)
print(lampl)
nup = 2*(nlevel[0]+nlevel[1])
if True:
    plt.plot(sig.spec)

    #plt.plot([0,sig.spec.size-1], [1e4, 1e4])
    #plt.plot([0,sig.spec.size-1], [nlevel[1], nlevel[1]])
    plt.plot([0,sig.spec.size-1], [nup,nup])

########### using list
self.Amp  [70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0]
(493.3477308788184, 1308.5750676328125)
[70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0]


/home/lio/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


# Experimental curve

In [58]:
#lnormden = [0]
lnormden = []
step = 10
for k in range(1,sig.spec.size//2, step):
    if k%(10*step) == 0:
        print(k)
    denoised = np.abs(np.fft.fft(urQRd(sig.fid, k, orda=sig.spec.size//2)))
    lnormden.append(norm(denoised))

# Analytic evolution

In [59]:
Ps = (sig.spec[sig.spec>nup]**2).sum()
Pss = (np.abs(np.fft.fft(urQRd(sig.fid, 15*nbpeaks, orda=sig.spec.size//2)))**2).sum()
print("Troncature ",Ps.real)
print("P_urQRd ", Pss)
print("Ps/P_urQRd {0} ".format(Ps/Pss))
s = nbpeaks
Pn = (sig.spec[sig.spec<nup]**2).sum()
n = lenfid//2-s

a,b,k,l = params(s,n,Ps,Pn)
ana = ANALYT()
#retriev_sig = np.sqrt(Ps*ana.sig**2 + Pn*ana.noise**2)
retriev_sig = np.sqrt(Ps)*ana.sig # + np.sqrt(Pn)*ana.noise
#plt.plot(retriev_sig,'--', label="analytical")
#plt.plot(ana.sig)
#plt.plot(ana.noise)

num = NUMER()
#retriev_sig = np.sqrt(Ps.real*num.sig**2 + Pn.real*num.noise**2)
retriev_sig = np.sqrt(Ps.real*num.sig**2 + Pn.real*(1-(1-num.noise)**2))
retriev_sig = np.sqrt(Pss.real*num.sig**2 +1.1*Pn*(1-((1-num.noise)**2)))
#retriev_sig = (np.sqrt(Pss.real)*num.sig + np.sqrt(Pn.real)*num.noise)*(np.sqrt(Pss+Pn)/(np.sqrt(Pss)+np.sqrt(Pn)))
#retriev_sig = np.sqrt(Ps.real*num.sig**3.3 + Pn.real*(1-(1-num.noise)**1.5))
#retriev_sig = np.sqrt(Ps.real*0.99*num.sig**2 + Pn.real*(1-1.1*(1-num.noise)**1.8))
#retriev_sig = (np.sqrt(Ps.real)*0.99*num.sig + np.sqrt(Pn.real*(1-1.1*(1-num.noise)**2)))*0.7
#retriev_sig = np.sqrt(Ps.real)*num.sig**2
#retriev_sig = np.sqrt(Ps.real*num.sig**2*0.9 + Pn.real*(1-(1-num.noise)**2)*0.9)
#retriev_sig = np.sqrt(Ps.real*num.sig**2*0.96 + Pn.real*(1-0.9*(1-num.noise)**2))
#retriev_sig = np.sqrt(Ps.real)*num.sig**2  + np.sqrt(Pn.real)*(1-(1-num.noise)**2)*0.3
#retriev_sig = np.sqrt(Ps.real)*num.sig**2  #+ np.sqrt(Pn.real)*num.noise
#retriev_sig = (Ps.real*num.sig**2 + Pn.real*(1-(1-num.noise)**2))*np.sqrt(Ps+Pn)/(Ps+Pn)
# 1-(1-num.noise)**2
#retriev_sig = np.sqrt(Ps.real*num.sig + Pn.real*num.noise)
#retriev_sig = np.sqrt(Ps).real*num.sig + np.sqrt(Pn).real*num.noise

plt.title("Comparisons analytique et expérimental")
plt.plot(lnormden, label="experimental")
plt.plot(retriev_sig[::step],'--', label="analytical")
plt.legend()

Troncature  97801484674.00461
P_urQRd  97010316082.80835
Ps/P_urQRd (1.0081555098791857+0j) 


/home/lio/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


# Signal

In [47]:
plt.plot(sig.spec)
plt.plot(np.abs(np.fft.fft(urQRd(sig.fid, 10*nbpeaks, orda=sig.spec.size//2))))

/home/lio/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [ ]:
np.sqrt(Ps.real)

In [35]:
plt.plot(lnormden)
plt.plot(retriev_sig[::step],'--', label="analytical")

/home/lio/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [ ]:
plt.plot(1-(1-num.noise)**2)
plt.plot(1-(1-ana.noise)**2)


In [ ]:
lnormden.insert(0, 0)

In [ ]:
lnormden[0]

In [ ]:
plt.plot(num.sig)
plt.plot(ana.sig)

In [ ]:
plt.plot(num.noise)
plt.plot(ana.noise)

In [ ]:
dir(lnormden)

In [ ]:
lnormden.remove(0)

In [ ]:
lnormden[0]